### Census API Requests
#### Authored By Clementine Sraha,  e-mail: clemiyarjah@gmail.com

In [1]:
#! pip install census

In [2]:
#!pip install us

In [3]:
#Import Dependencies
import numpy as np
import pandas as pd
import json
import requests
from census import Census
from us import states

In [4]:
#Census API key
from config import census_api
c=Census(census_api,year=2017)

In [5]:
state_population=c.acs5.get(('NAME', 'B01003_001E'), {'for': 'state:*'})

In [6]:
state_pop_df=pd.DataFrame(state_population)
state_pop_df=state_pop_df.rename(columns={'NAME':'State','B01003_001E':'Population',"state":"State_FIPS"})
state_pop_df = state_pop_df[state_pop_df.State != 'Puerto Rico']
state_pop_df.head(5)

,State,Population,State_FIPS
1,Alabama,4850771.0,01
2,Alaska,738565.0,02
3,Arizona,6809946.0,04
4,Arkansas,2977944.0,05
5,California,38982847.0,06


### Google Geocode API

In [7]:
from config import geocode_key

#### State Data

In [8]:
state_series=state_pop_df['State']
state_list=state_series.tolist()
#state_list

In [9]:
state_coords_df=pd.DataFrame(state_list,columns=['State'])

In [10]:
name=[]
lat=[]
long=[]

In [11]:
for i in state_list:
    target_url=(f"https://maps.googleapis.com/maps/api/geocode/json?address={i}&key={geocode_key}")
    geodata=requests.get(target_url).json()
    state_name=geodata['results'][0]['address_components'][0]['long_name']
    name.append(state_name)
    
    latitude=geodata['results'][0]['geometry']['location']['lat']
    lat.append(latitude)
    
    longitude=geodata['results'][0]['geometry']['location']['lng']
    long.append( longitude)

In [12]:
print(len(name))
print(len(lat))
print(len(long))

51
51
51


In [13]:
state_coords_df['Latitude']=pd.DataFrame(lat)
state_coords_df['Longitude']=pd.DataFrame(long)
state_coords_df.head(5)

,State,Latitude,Longitude
0,Alabama,32.318231,-86.902298
1,Alaska,64.200841,-149.493673
2,Arizona,34.048928,-111.093731
3,Arkansas,35.201050,-91.831833
4,California,36.778261,-119.417932


In [14]:
state_df=pd.merge(state_pop_df,state_coords_df, on='State')

In [15]:
state_df.head(5)

,State,Population,State_FIPS,Latitude,Longitude
0,Alabama,4850771.0,01,32.318231,-86.902298
1,Alaska,738565.0,02,64.200841,-149.493673
2,Arizona,6809946.0,04,34.048928,-111.093731
3,Arkansas,2977944.0,05,35.201050,-91.831833
4,California,38982847.0,06,36.778261,-119.417932


In [16]:
state_json=state_df.to_json(orient='records')
parsed=json.loads(state_json)


In [17]:
with open('state_pop.json', 'w') as f:
    json.dump(parsed, f, indent=4)

#### County Data

In [18]:
county_population=c.acs5.get(('NAME', 'B01003_001E'), {'for': 'county:*'})
county_population

[{'NAME': 'Corozal Municipio, Puerto Rico',
  'B01003_001E': 34933.0,
  'state': '72',
  'county': '047'},
 {'NAME': 'Maunabo Municipio, Puerto Rico',
  'B01003_001E': 11297.0,
  'state': '72',
  'county': '095'},
 {'NAME': 'Peñuelas Municipio, Puerto Rico',
  'B01003_001E': 21661.0,
  'state': '72',
  'county': '111'},
 {'NAME': 'Ponce Municipio, Puerto Rico',
  'B01003_001E': 148863.0,
  'state': '72',
  'county': '113'},
 {'NAME': 'San Sebastián Municipio, Puerto Rico',
  'B01003_001E': 38970.0,
  'state': '72',
  'county': '131'},
 {'NAME': 'Toa Alta Municipio, Puerto Rico',
  'B01003_001E': 74169.0,
  'state': '72',
  'county': '135'},
 {'NAME': 'Toa Baja Municipio, Puerto Rico',
  'B01003_001E': 81905.0,
  'state': '72',
  'county': '137'},
 {'NAME': 'Vieques Municipio, Puerto Rico',
  'B01003_001E': 8931.0,
  'state': '72',
  'county': '147'},
 {'NAME': 'Yabucoa Municipio, Puerto Rico',
  'B01003_001E': 35025.0,
  'state': '72',
  'county': '151'},
 {'NAME': 'Aguada Municipio, P

In [19]:
county_pop_df=pd.DataFrame(county_population)
county_pop_df=county_pop_df.rename(columns={'NAME':'County','B01003_001E':'Population',
                                            'state':'State_FIPS','county':'County_FIPS'})
county_pop_df = county_pop_df[county_pop_df.State_FIPS != '72']
county_pop_df.head(5)

,County,Population,State_FIPS,County_FIPS
78,"Monroe County, Alabama",21745.0,01,099
79,"Lawrence County, Alabama",33288.0,01,079
80,"Lee County, Alabama",156597.0,01,081
81,"Marion County, Alabama",30058.0,01,093
82,"Pickens County, Alabama",20170.0,01,107


In [20]:
county_series=county_pop_df['County']
county_list=county_series.tolist()
county_list

['Monroe County, Alabama',
 'Lawrence County, Alabama',
 'Lee County, Alabama',
 'Marion County, Alabama',
 'Pickens County, Alabama',
 'Sumter County, Alabama',
 'Jefferson County, Alabama',
 'Choctaw County, Alabama',
 'Franklin County, Alabama',
 'Marengo County, Alabama',
 'Russell County, Alabama',
 'Cherokee County, Alabama',
 'Covington County, Alabama',
 'Crenshaw County, Alabama',
 'Dallas County, Alabama',
 'Lauderdale County, Alabama',
 'Lowndes County, Alabama',
 'Macon County, Alabama',
 'Limestone County, Alabama',
 'Shelby County, Alabama',
 'St. Clair County, Alabama',
 'Winston County, Alabama',
 'Baldwin County, Alabama',
 'Cullman County, Alabama',
 'Elmore County, Alabama',
 'Houston County, Alabama',
 'Jackson County, Alabama',
 'Talladega County, Alabama',
 'Tuscaloosa County, Alabama',
 'Washington County, Alabama',
 'Clay County, Alabama',
 'Coffee County, Alabama',
 'Morgan County, Alabama',
 'Pike County, Alabama',
 'Chilton County, Alabama',
 'Colbert County,

In [21]:
county_coords_df=pd.DataFrame(county_list,columns=['County'])

In [22]:
lat=[]
long=[]

In [23]:
for i in county_list:
    target_url=(f"https://maps.googleapis.com/maps/api/geocode/json?address={i}&key={geocode_key}")
    geodata=requests.get(target_url).json()
    #print(geodata)
    latitude=geodata['results'][0]['geometry']['location']['lat']
    lat.append(latitude)
    longitude=geodata['results'][0]['geometry']['location']['lng']
    long.append( longitude)
 

In [24]:
county_coords_df['Latitude']=pd.DataFrame(lat)
county_coords_df['Longitude']=pd.DataFrame(long)
county_coords_df.head(5)

,County,Latitude,Longitude
0,"Monroe County, Alabama",31.511907,-87.460397
1,"Lawrence County, Alabama",34.567428,-87.301613
2,"Lee County, Alabama",32.588870,-85.396277
3,"Marion County, Alabama",34.044117,-87.776333
4,"Pickens County, Alabama",33.333995,-88.090076


In [25]:
county_df=pd.merge(county_pop_df,county_coords_df, on='County')

In [26]:
county_df.head(5)

,County,Population,State_FIPS,County_FIPS,Latitude,Longitude
0,"Monroe County, Alabama",21745.0,01,099,31.511907,-87.460397
1,"Lawrence County, Alabama",33288.0,01,079,34.567428,-87.301613
2,"Lee County, Alabama",156597.0,01,081,32.588870,-85.396277
3,"Marion County, Alabama",30058.0,01,093,34.044117,-87.776333
4,"Pickens County, Alabama",20170.0,01,107,33.333995,-88.090076


In [27]:
county_json=county_df.to_json(orient='records')
parsed=json.loads(county_json)

In [28]:
with open('county_pop.json', 'w') as f:
    json.dump(parsed, f, indent=4)